# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
#imports

from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import pandas as pd
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

## Workspace & Experiment

Connect to the workspace and create an Experiment

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'Hyperdrive-ML'

experiment=Experiment(ws, experiment_name)
experiment

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
quick-starts-ws-234821
aml-quickstarts-234821
westeurope
610d6e37-4747-4a20-80eb-3aad70a55f43


Name,Workspace,Report Page,Docs Page
Hyperdrive-ML,quick-starts-ws-234821,Link to Azure Machine Learning studio,Link to Documentation


#### Create or Attach an AmlCompute cluster

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded....................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


#### Dataset

In [25]:
dataset = Dataset.get_by_name(ws, name='mpg-pred')

#view the data
dataset.take(5).to_pandas_dataframe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year
0,18.0,8,307.0,130,3504,12.0,70
1,15.0,8,350.0,165,3693,11.5,70
2,18.0,8,318.0,150,3436,11.0,70
3,16.0,8,304.0,150,3433,12.0,70
4,17.0,8,302.0,140,3449,10.5,70


#### Enviroment Build & Logs

In [4]:

# Setup environment for your training run - Docker Enviroment with depandencies in .yml file
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#Build the Enviroment and confirm build
build = sklearn_env.build(workspace=ws)
build.wait_for_completion(show_output=True)

Building a non-registered environment is not supported. Registering environment.


Image Build Status: GettingOrProvisioningACR

Image Build Status: Queued

2023/06/01 18:36:40 Downloading source code...
2023/06/01 18:36:41 Finished downloading source code
2023/06/01 18:36:41 Creating Docker network: acb_default_network, driver: 'bridge'
2023/06/01 18:36:42 Successfully set up Docker network: acb_default_network
2023/06/01 18:36:42 Setting up Docker configuration...
2023/06/01 18:36:42 Successfully set up Docker configuration
2023/06/01 18:36:42 Logging in to registry: 7579fbbfa60040d19edf0ad43113cfa7.azurecr.io
2023/06/01 18:36:43 Successfully logged into 7579fbbfa60040d19edf0ad43113cfa7.azurecr.io
2023/06/01 18:36:43 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2023/06/01 18:36:43 Scanning for dependencies...
2023/06/01 18:36:43 Successfully scanned dependencies
2023/06/01 18:36:43 Launching container with name: acb_step_0
Sending build context to Docker daemon  71.68kB

Step 1/21 : FROM mcr.microsoft.com/

## Hyperdrive Configuration


In [5]:

# Specify parameter sampler -- Randomly sampling various Random Foresr Hyperparameter  
ps = RandomParameterSampling(
    {
        '--n_estimators' : choice(200, 300, 400),
        '--max_depth': choice(10, 20, 30, 40, 50),
        '--bootstrap': choice(1,0)
    }
)

# Specify a Policy -- Bandit policy for early termination
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")


# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='training',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env
                      )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name='r2_score',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_concurrent_runs=4,
                                     max_total_runs=16,                                     
                                    )

In [6]:
#Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_config) #use the experiment to submit the hyperdrive config



## Run Details


In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
# Monitor HyperDrive runs 
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_2743343d-e249-482d-b015-c37c75482dcf
Web View: https://ml.azure.com/runs/HD_2743343d-e249-482d-b015-c37c75482dcf?wsid=/subscriptions/610d6e37-4747-4a20-80eb-3aad70a55f43/resourcegroups/aml-quickstarts-234821/workspaces/quick-starts-ws-234821&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-01T18:43:40.171859][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-06-01T18:43:40.6219155Z][SCHEDULER][INFO]Scheduling job, id='HD_2743343d-e249-482d-b015-c37c75482dcf_0' 
[2023-06-01T18:43:40.7234389Z][SCHEDULER][INFO]Scheduling job, id='HD_2743343d-e249-482d-b015-c37c75482dcf_1' 
[2023-06-01T18:43:40.8792042Z][SCHEDULER][INFO]Scheduling job, id='HD_2743343d-e249-482d-b015-c37c75482dcf_2' 
[2023-06-01T18:43:40.9791192Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_2743343d-e249-482d-b015-c37c75482dcf_1' 
[2023-06-01T18:43:41.0210289Z][SCHEDULER][INFO]Scheduling job, id='HD_2743343d-e249-482d-b015-c37c75482dcf_3

{'runId': 'HD_2743343d-e249-482d-b015-c37c75482dcf',
 'target': 'cluster1',
 'status': 'Completed',
 'startTimeUtc': '2023-06-01T18:43:39.528456Z',
 'endTimeUtc': '2023-06-01T18:55:40.595216Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"r2_score","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'c855568e-631b-497d-84ba-888c325e963e',
  'user_agent': 'python/3.7.0 (Windows-10-10.0.22621-SP0) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': '30',
  'score': '0.9288660305402492',
  'best_child_run_id': 'HD_2743343d-e249-482d-b015-c37c75482dcf_11',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2743343d-e249-482d-b015-c37c75482dcf_11'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-tr

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:

# Get your best run and save the model from that run.



#Best Run and metrics
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())



print(best_run)


best run details : {'runId': 'HD_2743343d-e249-482d-b015-c37c75482dcf_11', 'target': 'cluster1', 'status': 'Completed', 'startTimeUtc': '2023-06-01T18:53:34.34744Z', 'endTimeUtc': '2023-06-01T18:53:50.750509Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': 'c855568e-631b-497d-84ba-888c325e963e', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--bootstrap', '1', '--max_depth', '20', '--n_estimators', '300'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'cluster1', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environm

In [10]:
#Register the Model
best_model = best_run.register_model(model_name = 'hyperdrive_best_run_mpg_pred', model_path = 'outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
service_name = 'hyperdrive-web-service'

inference_config = InferenceConfig(entry_script='score.py', environment=sklearn_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=8)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[best_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-06-01 15:22:45-04:00 Creating Container Registry if not exists.
2023-06-01 15:22:45-04:00 Registering the environment.
2023-06-01 15:22:46-04:00 Use the existing image.
2023-06-01 15:22:47-04:00 Generating deployment configuration.
2023-06-01 15:22:48-04:00 Submitting deployment to compute.
2023-06-01 15:22:54-04:00 Checking the status of deployment hyperdrive-web-service..
2023-06-01 15:24:39-04:00 Checking the status of inference endpoint hyperdrive-web-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


Test API Endpoints

In [12]:
## Test endpoints - API request

import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "data": [
    {
        "cylinders": 6,
        "displacement": 320.0,
        "horsepower": 160,
        "weight": 3000,
        "acceleration": 15.0,
        "model year": 70
    }
  ]
}

body = str.encode(json.dumps(data))

url = 'http://cadb9776-ffb3-4195-89b7-f82a7912df13.westeurope.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[15.732666666666665]'


TODO: In the cell below, print the logs of the web service and delete the service

In [13]:
print(service.get_logs())

2023-06-01T19:24:21,934701200+00:00 - rsyslog/run 
2023-06-01T19:24:21,934669300+00:00 - iot-server/run 
2023-06-01T19:24:21,942718300+00:00 - gunicorn/run 
2023-06-01T19:24:21,945538000+00:00 | gunicorn/run | 
2023-06-01T19:24:21,948888300+00:00 | gunicorn/run | ###############################################
2023-06-01T19:24:21,954141000+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-06-01T19:24:21,958017400+00:00 | gunicorn/run | ###############################################
2023-06-01T19:24:21,958314600+00:00 - nginx/run 
2023-06-01T19:24:21,963086700+00:00 | gunicorn/run | 
2023-06-01T19:24:21,966881500+00:00 | gunicorn/run | 
2023-06-01T19:24:21,980026200+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230120.v2
2023-06-01T19:24:21,983374300+00:00 | gunicorn/run | 
2023-06-01T19:24:21,987284400+00:00 | gunicorn/run | 
2023-06-01T19:24:21,991806400+00:00 | gunicorn/run | PATH environment variable: /azureml-env

Delete all resources Not used

In [35]:
# Delete all resources not used
service.delete()
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

